In [ ]:
import os
import sys
os.chdir("../")
sys.path.append("../")

import re
import logging
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
import time
import pickle
from thop import profile
from collections import defaultdict
from data.configs.demo import config
from collections import defaultdict

from transformers import AutoTokenizer, AutoModel, BertModel, BertConfig, AutoModelForSequenceClassification
from utils.Manager import Manager

from models.Embeddings.BERT import BERT_Embedding
from models.Encoders.CNN import CNN_Encoder,CNN_User_Encoder
from models.Encoders.RNN import RNN_Encoder,RNN_User_Encoder
from models.Encoders.MHA import MHA_Encoder, MHA_User_Encoder
from models.Modules.DRM import Matching_Reducer, Identical_Reducer
from models.Rankers.BERT import BERT_Onepass_Ranker, BERT_Original_Ranker
from models.Rankers.CNN import CNN_Ranker
from models.Encoders.Pooling import Attention_Pooling, Average_Pooling
from models.UniLM.modeling import TuringNLRv3Model, TuringNLRv3ForSequenceClassification, relative_position_bucket
from models.UniLM.configuration_tnlrv3 import TuringNLRv3Config

from models.TwoTowerBaseModel import TwoTowerBaseModel

from models.Encoders.BERT import BERT_Encoder
from models.Encoders.Pooling import *

from models.TESRec import TESRec
from models.XFormer import XFormer
from models.PLM import PLM
 
from models.Modules.Attention import MultiheadAttention, get_attn_mask, XSoftmax
# torch.set_printoptions(threshold=100000)

In [ ]:
from transformers import LongformerConfig, LongformerModel, BigBirdModel, BigBirdConfig

In [46]:
m.config.is_decoder

False

In [42]:
a = BigBirdConfig()
a.block_size = 2
a.num_random_blocks = 2
a.max_position_embeddings = 64
m = BigBirdModel(a)

In [35]:
inputs = t3("i",padding="max_length",max_length=32,return_tensors='pt')

In [43]:
res = m(**inputs)

In [37]:
m7.config, m.config

(BigBirdConfig {
   "_name_or_path": "google/bigbird-roberta-base",
   "architectures": [
     "BigBirdForPreTraining"
   ],
   "attention_probs_dropout_prob": 0.1,
   "attention_type": "block_sparse",
   "block_size": 64,
   "bos_token_id": 1,
   "classifier_dropout": null,
   "eos_token_id": 2,
   "gradient_checkpointing": false,
   "hidden_act": "gelu_new",
   "hidden_dropout_prob": 0.1,
   "hidden_size": 768,
   "initializer_range": 0.02,
   "intermediate_size": 3072,
   "layer_norm_eps": 1e-12,
   "max_position_embeddings": 4096,
   "model_type": "big_bird",
   "num_attention_heads": 12,
   "num_hidden_layers": 12,
   "num_random_blocks": 3,
   "pad_token_id": 0,
   "position_embedding_type": "absolute",
   "rescale_embeddings": false,
   "sep_token_id": 66,
   "transformers_version": "4.10.0",
   "type_vocab_size": 2,
   "use_bias": true,
   "use_cache": true,
   "vocab_size": 50358
 },
 BigBirdConfig {
   "attention_probs_dropout_prob": 0.1,
   "attention_type": "block_sparse",


In [ ]:
# m = AutoModel.from_pretrained('bert-base-uncased',cache_dir=config.path + 'bert_cache/')
# m2 = AutoModel.from_pretrained('microsoft/deberta-base',cache_dir=config.path + 'bert_cache/')
# m3 = TuringNLRv3ForSequenceClassification.from_pretrained(config.unilm_path, config=TuringNLRv3Config.from_pretrained(config.unilm_config_path))
# m4 = AutoModel.from_pretrained('google/reformer-crime-and-punishment', cache_dir=config.path + "bert_cache/")
# m5 = AutoModel.from_pretrained('funnel-transformer/small-base', cache_dir=config.path + "bert_cache/")
# m6 = AutoModel.from_pretrained('distilbert-base-uncased',cache_dir=config.path + 'bert_cache/')
m7 = AutoModel.from_pretrained("google/bigbird-roberta-base", cache_dir=config.path + "bert_cache/")
# m8 = AutoModel.from_pretrained("allenai/longformer-base-4096", cache_dir=config.path + "bert_cache/")

# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")
# t3 = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', cache_dir=config.path + "bert_cache/")
# t4 = AutoTokenizer.from_pretrained('google/reformer-crime-and-punishment', cache_dir=config.path + "bert_cache/")
# t5 = AutoTokenizer.from_pretrained('funnel-transformer/small-base', cache_dir=config.path + "bert_cache/")
# t6 = AutoTokenizer.from_pretrained('distilbert-base-uncased', cache_dir=config.path + "bert_cache/")
# t7 = AutoTokenizer.from_pretrained("google/bigbird-roberta-base", cache_dir=config.path + "bert_cache/")

In [ ]:
config.bert = 'bert'
config.mode = "encode"
config.scale = "large"

manager = Manager(config)

loaders = manager.prepare()
X1 = list(loaders[0])
# X2 = list(loaders[1])
x1 = X1[0]
# x2 = X2[0]

In [ ]:
# embedding = BERT_Embedding(manager)
manager.hidden_dim = 768
manager.reducer = "none"
# encoderN = CNN_Encoder(manager)
# encoderN = RNN_Encoder(manager)
# encoderN = MHA_Encoder(manager)

# encoderU = CNN_User_Encoder(manager)
encoderU = RNN_User_Encoder(manager)
# encoderU = MHA_User_Encoder(manager)
# encoderU = Attention_Pooling(manager)
# encoderU = Average_Pooling(manager)

# reducer = Matching_Reducer(manager)
# reducer = Identical_Reducer(manager)

# ranker = CNN_Ranker(manager)
# ranker = BERT_Onepass_Ranker(manager)
# ranker = BERT_Original_Ranker(manager)

# model = TESRec(manager, embedding, encoderN, encoderU, reducer).to(manager.device)
# model = ESM(manager, embedding, encoderN, encoderU, reducer, ranker).to(manager.device)
model = PLM(manager, encoderU).to(manager.device)

In [ ]:
# GateFormer
embedding = 50 * 2 * 30 * 768 * 2

ext_encoding = 50*(3*150*768*30 + 30*150*4) + 8*150*150*50
reduction = (2*30*150 + 30*math.log2(30))*50 + 50*3

bert_embed = 150 * 2 * 768 * 2
bert_project = 150*12*64*2
bert_attn = 150*12*64*150*2 + 12*150*150*64*2
bert_intm = 150*768*768*2 + 150*768*2 + 150*768*3072*4
bert_pool = 150*768*4

sum = embedding + ext_encoding + reduction + bert_embed + (bert_project + bert_attn + bert_intm) * 12 + bert_pool
sum

In [ ]:
# GateFormer First
embedding = 50 * 2 * 30 * 768 * 2

bert_embed = 150 * 2 * 768 * 2
bert_project = 150*12*64*2
bert_attn = 150*12*64*150*2 + 12*150*150*64*2
bert_intm = 150*768*768*2 + 150*768*2 + 150*768*3072*4
bert_pool = 150*768*4

sum = embedding + bert_embed + (bert_project + bert_attn + bert_intm) * 12 + bert_pool
sum

In [ ]:
# GateFormer BERT
embedding = 50 * 2 * 30 * 768 * 2

ext_encoding = 50*30*768*768*2*3 + 50*30*768*30*2*2 + 50*30*768*2*2
reduction = (2*30*150 + 30*math.log2(30))*50 + 50*3

bert_embed = 150 * 2 * 768 * 2
bert_project = 150*12*64*2
bert_attn = 150*12*64*150*2 + 12*150*150*64*2
bert_intm = 150*768*768*2 + 150*768*2 + 150*768*3072*4
bert_pool = 150*768*4

sum = embedding + ext_encoding + reduction + bert_embed + (bert_project + bert_attn + bert_intm) * 12 + bert_pool
sum

In [ ]:
# GateFormer ATT
import math
embedding = 50 * 2 * 30 * 768 * 2

ext_encoding = 50*(3*150*768*30 + 30*150*4) + 50*150*4
reduction = (2*30*150 + 30*math.log2(30))*50 + 50*3

bert_embed = 50*3*768*2*2
bert_project = 50*12*3*64*2*3
bert_attn = 50*12*3*64*3*2 + 50*12*3*3*64*2
bert_intm = 50*3*768*768*2 + 50*3*768*2 + 50*3*768*3072*4
bert_pool = 50*1*768*768*2

bert = ext_encoding + reduction + (bert_project + bert_attn + bert_intm) * 12 + bert_pool

rnn = 8*2*768*768*50

sum = embedding + bert + rnn
sum

In [ ]:
# GateFormer
embedding = 50 * 2 * 100 * 768 * 2

bert_embed = 150 * 2 * 768 * 2
bert_project = 150*12*64*2
bert_attn = 150*12*64*150*2 + 12*150*150*64*2
bert_intm = 150*768*768*2 + 150*768*2 + 150*768*3072*4
bert_pool = 150*768*4

sum = embedding + bert_embed + (bert_project + bert_attn + bert_intm) * 12 + bert_pool
sum

In [ ]:
# NRMS
embedding = 50 * 2 * 30 * 300 * 2

project = 50 * 30 * 300 * 256 * 2 * 3
attn = 50*30*256*30*4
attnA = 50*30*256*2

project2 = 50*256*256*2*3
attn2 = 50*256*50*4
attnA2 = 50*256*2

sum = embedding + project + project2 + attn + attn2 + attnA + attnA2
sum

In [ ]:
# LSTUR
embedding = 50 * 2 * 30 * 300 * 2

encoding = 3*300*150*30*50
attn=50*30*150*2 + 50*30*150*2
lstm = 50*150*150*8*2

sum = embedding + encoding + lstm + attn
sum

In [ ]:
# NAML
embedding = 50 * 2 * 30 * 300 *2 *2
encoding = 3*300*150*30*50*2
attn=(50*30*150*2 + 50*30*150*2)*2
attn2=50*150*2 + 50*150*2
sum = embedding + encoding + attn + attn2
sum

In [ ]:
# BigBird
embedding = 2 * 1024 * 768 * 2 * 3

bert_project = 1024*12*64*2
bert_attn = 1024*12*64*2 + 12*1024*64*2
bert_intm = 1024*768*768*2 + 1024*768*2 + 1024*768*3072*4
bert_pool = 1024*768*4

sum = embedding + (bert_project + bert_attn + bert_intm) * 12 + bert_pool
sum

In [ ]:
# longformer
embedding = 2 * 1024 * 768 * 2 * 3

bert_project = 1024*12*64*2
bert_attn = 1024*512*12*64*2 + 12*1024*512*64*2
bert_intm = 1024*768*768*2 + 1024*768*2 + 1024*768*3072*4
bert_pool = 1024*768*4

sum = embedding + (bert_project + bert_attn + bert_intm) * 12 + bert_pool
sum

In [ ]:
# EBNR
embedding = 50 * 2 * 30 * 768 * 2 * 3

bert_project = 50*12*30*64*2*3
bert_attn = 50*12*30*64*30*2 + 50*12*30*30*64*2
bert_intm = 50*30*768*768*2 + 50*30*768*2 + 50*30*768*3072*4
bert_pool = 50*1*768*768*2

bert = (bert_project + bert_attn + bert_intm) * 12 + bert_pool

rnn = 8*2*768*768*50

sum = embedding + bert + rnn
sum

In [ ]:
# Synthesizer
embedding = 50 * 2 * 30 * 768 * 2 * 3

bert_project = 50*12*30*64*2*3
bert_attn = 50*12*30*64*30*2 + 50*12*30*30*64*2
bert_intm = 50*30*768*768*2 + 50*30*768*2 + 50*30*768*3072*4
bert_pool = 50*1*768*768*2

bert = (bert_project + bert_attn + bert_intm) * 12 + bert_pool

rnn = 8*2*768*768*50

sum = embedding + bert + rnn
sum

In [ ]:
# funnel
embedding = 50 * 2 * 30 * 768 * 2 * 3

bert_project = 50*12*30*64*2*3
bert_attn = 50*12*30*64*30*2 + 50*12*30*30*64*2
bert_intm = 50*30*768*768*2 + 50*30*768*2 + 50*30*768*3072*4
bert_pool = 50*1*768*768*2

bert = (bert_project + bert_attn + bert_intm) * 10 + bert_pool

rnn = 8*2*768*768*50

sum = embedding + bert + rnn
sum

In [ ]:
# newbert
embedding = 50 * 2 * 30 * 768 * 2 * 3

bert_project = 50*12*30*64*2*3
bert_attn = 50*12*30*64*30*2 + 50*12*30*30*64*2
bert_intm = 50*30*768*768*2 + 50*30*768*2 + 50*30*768*3072*4
bert_pool = 50*1*768*768*2

bert = (bert_project + bert_attn + bert_intm) * 4 + bert_pool

rnn = 8*2*768*768*50

sum = embedding + bert + rnn
sum

In [ ]:
# distilbert
embedding = 50 * 2 * 30 * 768 * 2 * 2

bert_project = 50*12*30*64*2*3
bert_attn = 50*12*30*64*30*2 + 50*12*30*30*64*2
bert_intm = 50*30*768*768*2 + 50*30*768*2 + 50*30*768*3072*4
bert_pool = 50*1*768*768*2

bert = (bert_project + bert_attn + bert_intm) * 6 + bert_pool

rnn = 8*2*768*768*50

sum = embedding + bert + rnn
sum

In [ ]:
189 / 255560 * 1000